In [1]:
import pandas as pd
import os
import numpy as np
import re
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Load data

In [2]:
data = pd.read_csv(os.path.join('..', 'data', 'Devices_Specs.csv'))

def inspect_data(data):
    dataCols = data.columns
    NullSum = data.isnull().sum()
    NullPercent = NullSum/len(data)*100
    UniqueSum = data.nunique()
    ColType = data.dtypes
    # nsamples = 5
    # Samples = [', '.join([str(data[col][i]) for i in range(nsamples)]) for col in data.columns]
    n_UniqueSample = 5
    UniqueSamples = [', '.join([str(data[col].unique()[i]) for i in range(n_UniqueSample if len(
        data[col].unique()) > n_UniqueSample else len(data[col].unique()))]) for col in data.columns]

    Inspec = pd.DataFrame({'Column': dataCols, 'NullSum': NullSum, 'NullPercent%': NullPercent,
                          'UniqueSum': UniqueSum, 'Type': ColType, 'Unique Sample': UniqueSamples})
    Inspec.index = [i for i in range(1, len(data.columns)+1)]
    return Inspec

# Lay cac cot thuoc MEMORY
memoryCols = [col for col in data.columns if re.search('MEMORY', col)]

# inspect data cac cot thuoc DISPLAY
memoryData = data[memoryCols]
inspect_data(memoryData)

,Column,NullSum,NullPercent%,UniqueSum,Type,Unique Sample
1,MEMORY_Card_slot,0,0.000000,158,object,"microSDXC (dedicated slot), microSDXC (uses shared SIM slot), microSDXC, microSDHC (dedicated slot), microSDHC (uses shared SIM slot)"
2,MEMORY_Internal,1792,14.321106,1136,object,"32GB 4GB RAM, 16GB 2GB RAM, 32GB 2GB RAM, 32GB 3GB RAM, 8GB 1GB RAM, 16GB 2GB RAM, 32GB 2GB RAM, 64GB 2GB RAM"


# Card_slot Col

Không có null

In [4]:
print(len(memoryData['MEMORY_Card_slot'].unique()))
memoryData['MEMORY_Card_slot'].value_counts()

158


No                                                                                                      3143
microSDHC (dedicated slot)                                                                              2968
microSDXC (dedicated slot)                                                                              2001
microSDXC (uses shared SIM slot)                                                                         979
microSDXC                                                                                                853
microSD (dedicated slot)                                                                                 815
microSDHC                                                                                                702
microSDHC (dedicated slot), 2 GB included                                                                166
miniSD                                                                                                   108
NM (Nano Memory), u

In [ ]:
def  unifyCardSlot(data: pd.DataFrame):
    data['MEMORY_Card_slot'] = data['MEMORY_Card_slot'].str.lower()
    for i in range(len(data['MEMORY_Card_slot'])):
        if '(uses shared sim slot)' in data['MEMORY_Card_slot'][i]:
        #     data['MEMORY_Card_slot'][i] = 'microsdxc'
        
        # elif data['MEMORY_Card_slot'][i] == 'microsdhc':
        #     data['MEMORY_Card_slot'][i] = 'microsd'
        

# Internal Col